# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846864154879                   -0.70    5.0         
  2   -7.852321021414       -2.26       -1.53    1.0   33.4ms
  3   -7.852614986598       -3.53       -2.56    1.5   37.5ms
  4   -7.852646027121       -4.51       -2.90    2.5   46.5ms
  5   -7.852646525509       -6.30       -3.21    1.0   34.7ms
  6   -7.852646679051       -6.81       -4.01    1.0   34.0ms
  7   -7.852646686225       -8.14       -5.34    1.2   38.7ms
  8   -7.852646686725       -9.30       -5.50    3.0   51.2ms
  9   -7.852646686730      -11.31       -6.21    1.0   34.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846811416770                   -0.70           4.5         
  2   -7.852523999294       -2.24       -1.64   0.80    2.2    425ms
  3   -7.852635667477       -3.95       -2.74   0.80    1.0   32.3ms
  4   -7.852646496876       -4.97       -3.25   0.80    2.2   43.2ms
  5   -7.852646673484       -6.75       -4.06   0.80    1.2   34.8ms
  6   -7.852646686365       -7.89       -4.79   0.80    1.8   39.5ms
  7   -7.852646686724       -9.44       -5.63   0.80    1.8   42.8ms
  8   -7.852646686730      -11.24       -6.96   0.80    2.2   43.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.400390e+01     3.442336e+00
 * time: 0.07780098915100098
     1     1.232377e+00     1.849758e+00
 * time: 0.38660287857055664
     2    -1.308605e+00     2.251410e+00
 * time: 0.421875
     3    -3.614169e+00     1.812183e+00
 * time: 0.4751889705657959
     4    -4.899624e+00     1.750819e+00
 * time: 0.5276849269866943
     5    -6.595659e+00     1.370181e+00
 * time: 0.5798280239105225
     6    -7.331069e+00     6.429945e-01
 * time: 0.6312530040740967
     7    -7.638945e+00     2.968303e-01
 * time: 0.667639970779419
     8    -7.740198e+00     1.753960e-01
 * time: 0.7030079364776611
     9    -7.792563e+00     1.590980e-01
 * time: 0.7385590076446533
    10    -7.819281e+00     7.705658e-02
 * time: 0.7745740413665771
    11    -7.836094e+00     6.813393e-02
 * time: 0.8106980323791504
    12    -7.842581e+00     5.806714e-02
 * time: 0.8463790416717529
    13    -7.845567e+00     3.909951e-02
 * time: 0.8818018436431885
  

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846699291812                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645739173                   -1.64         
  2   -7.852646686730       -6.02       -3.70    2.67s
  3   -7.852646686730      -12.95       -7.11    262ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 8.032843069880328e-7
|ρ_newton - ρ_scfv| = 1.21564928605517e-7
|ρ_newton - ρ_dm|   = 2.4322996018631903e-9
